<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-10-15 11:27:53--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  24.2MB/s    in 58s     

2022-10-15 11:28:53 (23.9 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# Посмотрим, есть ли вообще слово cat в wv_embeddings
wv_embeddings.__contains__('cat')

True

In [ ]:
'''your code'''
flag = True
for word, sim in wv_embeddings.most_similar(positive=['dog'], topn=5):
  if word == 'cat':
    flag = False
    print('Слово cat входит в топ-5 близких слов к слову dog')
if flag:
  print(f"Слово cat не входит в топ-5 близких слов к слову dog и находится на {wv_embeddings.rank('dog', 'cat')} на месте")

Слово cat не входит в топ-5 близких слов к слову dog и находится на 26 на месте


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

# Попробуем 2 разных токенизатора
class MyreTokenizer:
    def __init__(self):
        pass
    def get_tokens(self, text, lower=True):
      if lower:
        text = text.lower()
      return re.findall('\w+', text)
      
retokenizer = MyreTokenizer()

In [ ]:
# Используем  WordPunctTokenizer
import numpy as np
from nltk.tokenize import WordPunctTokenizer

class MyTokenizer(WordPunctTokenizer):
  def __init__(self):
    super().__init__()
  def get_tokens(self, text, lower=True):
    if lower:
      text = text.lower()
    return self.tokenize(text)

tokenizer = MyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200, lower=False):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    sum_question_vec = np.zeros(dim)
    cnt_token = 0

    tokens = tokenizer.get_tokens(question, lower)
    
    for token in tokens:
      if embeddings.__contains__(token):
        sum_question_vec += embeddings[token]
        cnt_token += 1        
        
    if cnt_token:
      sum_question_vec /= cnt_token
    
    return  sum_question_vec

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
'''your code'''
question = 'I love neural networks'
print('Токенизатор, основанный на регулярных выражениях')
print(f'Компонента с индексом 2 без перехода в нижний регистр = {round(question_to_vec(question, wv_embeddings, retokenizer, lower=False)[2], 2)}')
print(f'Компонента с индексом 2 c переходом в нижний регистр = {round(question_to_vec(question, wv_embeddings, retokenizer, lower=True)[2], 2)}')

Токенизатор, основанный на регулярных выражениях
Компонента с индексом 2 без перехода в нижний регистр = -1.29
Компонента с индексом 2 c переходом в нижний регистр = -1.29


In [ ]:
'''your code'''
question = 'I love neural networks'
print('Токенизатор, основанный на WordPunctTokenizer')
print(f'Компонента с индексом 2 без перехода в нижний регистр = {round(question_to_vec(question, wv_embeddings, tokenizer, lower=False)[2], 2)}')
print(f'Компонента с индексом 2 c переходом в нижний регистр = {round(question_to_vec(question, wv_embeddings, tokenizer, lower=True)[2], 2)}')

Токенизатор, основанный на WordPunctTokenizer
Компонента с индексом 2 без перехода в нижний регистр = -1.29
Компонента с индексом 2 c переходом в нижний регистр = -1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

#### **Ответ** 
Коротко: 1

Подробно:

Максимальное значение выражения `Hits@47 - DCG@1` будет достигаться при максимальном `Hits@47` и минимальном `DCG@1`.

**Hits@47** $K = 47$

Рассмотрим раздельно каждую из составляющих:
- $[rank\_q_i^{'} \le 47]$ Индикаторная функция, принимает 2 значения: 0 и 1. Ее максимум будет в 1.
- $\sum_{i=1}^N \,$. Сумма индикаторных функций для всех вопросов. Максимум ее будет достигаться в том случае, если при всех $N$ вопросах позиция дубликата будет $\leq 47$.
- $\frac{1}{N}$ Усреднение на количество вопросов.

Т.е., если у нас было N вопросов, и во всех случаях ранк дубликата был меньше 47, тогда получится:
$$ \frac{1}{N} * (N * 1) = 1$$
Какое бы количество вопросов ни было выбрано, максимальное значение Hits@47 не будет превышать 1.

**DCG@1** $K = 1$

Рассмотрим подробнее:
- $\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot [rank\_q_i^{'} \le 1]$ 
  - При $rank\_q_i^{'} = 1$ выражение $=1$, 
  - При $rank\_q_i^{'} > 1$ выражение $=0$
  - Следовательно, минимум достигается в 0.

Все остальные составляющие выражения не могут перевести значение выражения в область отрицательных значений. А значит, минимум DCG@1 = 0.

Следовательно, максимум `Hits@47 - DCG@1` = $1 - 0 = 1$

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

#### **Ответ**

[K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}}$


In [ ]:
import numpy as np
print('DCG@10 при rank = 9 составляет:', round(1 / np.log2(10), 1))

DCG@10 при rank = 9 составляет: 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
import numpy as np

def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    dup = np.array(dup_ranks)
    hits_value = sum(dup <= k) / len(dup)

    return hits_value    

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    
    dup = np.array(dup_ranks)
    dup_log = 1 / np.log2(1 + dup)
    dup_ind = (dup <= k).astype(int)

    dcg_value = sum(dup_log * dup_ind) / len(dup)

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
# !unzip stackoverflow_similar_questions.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/DLS_NLP/1w_simple_embeddings/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/DLS_NLP/1w_simple_embeddings/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200, lower=False):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    question_copy = deepcopy(question)
    main_vec = question_to_vec(question_copy, embeddings, tokenizer, dim=200, lower=lower)
    lst_candidates = []
    for ind, val in enumerate(candidates):
      candidate_copy = deepcopy(val)
      candidate_vec = question_to_vec(candidate_copy, embeddings, tokenizer, dim=200, lower=lower)
      metric = cosine_similarity(main_vec.reshape(1,-1), candidate_vec.reshape(1,-1))
      lst_candidates.append((ind, val, metric))

    lst_candidates.sort(key=lambda x: 1-x[2])
    result = [(tup[0], tup[1]) for tup in lst_candidates]    

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
print('Токенизатор, основанный на регулярных выражениях')
print('Результат без перехода в нижний регистр')
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, retokenizer, lower=False)
        print(ranks)
        print()
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('Результат с переходом в нижний регистр')
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, retokenizer, lower=True)
        print(ranks)
        print()
print('_________________________________________________________________________________________________________________________________________________________________________')
print('Токенизатор, основанный на WordPunctTokenizer')
print('Результат без перехода в нижний регистр')
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, lower=False)
        print(ranks)
        print()
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print('Результат с переходом в нижний регистр')
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer, lower=True)
        print(ranks)
        print()

Токенизатор, основанный на регулярных выражениях
Результат без перехода в нижний регистр
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Результат с переходом в нижний регистр
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]

_________________________

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

#### Ответ:
* Без перехода в нижний регистр (as is): 102
* С переходом в нижний регистр: 021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

**Токенизатор, основанный на WordPunctTokenizer**

In [ ]:
# Без переxода в нижний регистр (as is)
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer, lower=False)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.386
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.875
DCG@1000: 0.434 | Hits@1000: 1.000


In [ ]:
# C переходом в нижний регистр
wv_ranking_lower = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer, lower=True)
    wv_ranking_lower.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_lower, k), k, hits_count(wv_ranking_lower, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.500 | Hits@   5: 0.580
DCG@  10: 0.521 | Hits@  10: 0.645
DCG@ 100: 0.568 | Hits@ 100: 0.875
DCG@ 500: 0.581 | Hits@ 500: 0.973
DCG@1000: 0.583 | Hits@1000: 1.000


**Токенизатор, основанный на регулярных выражениях**

In [ ]:
# Без переxода в нижний регистр (as is)
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, retokenizer, lower=False)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


In [ ]:
# C переходом в нижний регистр
wv_ranking_lower = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, retokenizer, lower=True)
    wv_ranking_lower.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_lower, k), k, hits_count(wv_ranking_lower, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


**Заключение**: 
- Лучшее качество показал токенизатор, основанный на регулярных выражениях, и переход в нижний регистр
- Токенизатор, основанный на регулярных выражениях убирает знаки препинания, в отличие от WordPunctTokenizer. Похоже, что наличие знаков препинания в данной задаче мешает.
- В данном случае переход в нижний регистр повышает качество. 

### Эмбеддинги, обученные на корпусе похожих вопросов

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
def glue_read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        lst_line = line.strip().split('\t')
        line_line = " ".join(lst_line)
        data.append([line_line])
    return data

In [ ]:
train_data = glue_read_corpus('./data/train.tsv')

In [ ]:
print('Размер train_data', len(train_data))

Размер train_data 1000000


In [ ]:
# Размер и содержание 5 первых строк
for i in range(5):
    print(i + 1, len(train_data[i]))
    print(train_data[i])

1 1
['converting string to list Convert Google results object (pure js) to Python object']
2 1
['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?']
3 1
['Sending array via Ajax fails Getting all list items of an unordered list in PHP']
4 1
['How to insert CookieCollection to CookieContainer? C# create cookie from string and send it']
5 1
['Updating one element of a bound Observable collection WPF- How to update the changes in list item of a list']


In [ ]:
# Посмотрим на максимальную длину предложений
max_len = 0
max_ind = None
for ind, sen in enumerate(train_data):
  if len(sen[0]) > max_len:
    max_len = len(sen[0])
    max_ind = ind 

print('Максимальная длина склеенных предложений:', max_len)

Максимальная длина склеенных предложений: 4007


In [ ]:
train_data[max_ind]

['Php array_push() vs myArray[] Double not (!!) operator in PHP What does "&" mean in this case? Where can I read about conditionals done with "?" and ":" (colon)? Should I use @ in my PHP code? What does the PHP syntax $var1->$var2 mean? What does the percent sign mean in PHP? What Does Using A Single Pipe \'|\' In A Function Argument Do? Reference assignment operator in PHP, =& What\'s the difference between :: (double colon) and -> (arrow) in PHP? In PHP, what does "<<<" represent? Type-juggling and (strict) greater/lesser-than comparisons in PHP What does the "&" sign mean in PHP? Where do we use the object operator "->" in PHP? PHP equivalent for Ruby\'s or-equals (foo ||=bar)? PHP - and / or keywords Using PHP 5.3 ?: operator What are the backticks `` called? What is the difference between single-quoted and double-quoted strings in PHP? What is the difference between <> and != What exactly does || mean? What does it mean to start a PHP function with an ampersand? The 3 different 

Это определенно не похоже на 2 склеенных вопроса

In [ ]:
'''your code'''

# Токенизатор, основанный на регулярных выражениях
re_words = []
for sample in train_data:
    q_tokens = retokenizer.get_tokens(sample[0], lower=True)
    re_words.append(q_tokens)

# Посмотрим первые 5 строк пар списков токенов
print('Токенизатор, основанный на регулярных выражениях')
for i in range(5):
    print(re_words[i])
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

# Токенизатор, основанный на WordPunctTokenizer
words = []
for sample in train_data:
    q_tokens = tokenizer.get_tokens(sample[0], lower=True) 
    words.append(q_tokens)
    
print('Токенизатор, основанный на WordPunctTokenizer')
# Посмотрим первые 5 строк пар списков токенов
for i in range(5):
    print(words[i])

Токенизатор, основанный на регулярных выражениях
['converting', 'string', 'to', 'list', 'convert', 'google', 'results', 'object', 'pure', 'js', 'to', 'python', 'object']
['which', 'html', '5', 'canvas', 'javascript', 'to', 'use', 'for', 'making', 'an', 'interactive', 'drawing', 'tool', 'event', 'handling', 'for', 'geometries', 'in', 'three', 'js']
['sending', 'array', 'via', 'ajax', 'fails', 'getting', 'all', 'list', 'items', 'of', 'an', 'unordered', 'list', 'in', 'php']
['how', 'to', 'insert', 'cookiecollection', 'to', 'cookiecontainer', 'c', 'create', 'cookie', 'from', 'string', 'and', 'send', 'it']
['updating', 'one', 'element', 'of', 'a', 'bound', 'observable', 'collection', 'wpf', 'how', 'to', 'update', 'the', 'changes', 'in', 'list', 'item', 'of', 'a', 'list']
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Токенизатор, основанный на WordPunctTokenizer
['conver

In [ ]:
len(words)

1000000

**1 итерация**

- Токенизатор, основанный на регулярных выражениях
- Переход в нижний регистр 
- window=5
- min_count=5

**Про выбор window**:
В [статье](https://habr.com/ru/post/446530/) затрагивается тема выбора окна. По умолчанию в Word2Vec window = 5. Поскольку мы имеем дело с короткими предложениями, считаю целесообразным в качестве первого эксперимента попробовать параметр по умолчанию.

min_count также по умолчанию равен 5 ([см. документацию](https://radimrehurek.com/gensim/models/word2vec.html?highlight=word2vec#module-gensim.models.word2vec)), для первого эксперимента не будем устанавливать другие значения.

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(re_words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5,
                 compute_loss=True)

embeddings_trained = model.wv
print('Latest training loss', model.get_latest_training_loss())        

Latest training loss 31364196.0


In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, retokenizer, lower=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.315 | Hits@   1: 0.315
DCG@   5: 0.390 | Hits@   5: 0.456
DCG@  10: 0.416 | Hits@  10: 0.538
DCG@ 100: 0.468 | Hits@ 100: 0.793
DCG@ 500: 0.487 | Hits@ 500: 0.942
DCG@1000: 0.494 | Hits@1000: 1.000


**2 итерация**

- Токенизатор, основанный на регулярных выражениях
- Переход в нижний регистр 
- window=7
- min_count=5

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(re_words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=7,
                 compute_loss=True)

embeddings_trained = model.wv

In [ ]:
print('Latest training loss', model.get_latest_training_loss()) 

Latest training loss 31521410.0


In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, retokenizer, lower=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.323 | Hits@   1: 0.323
DCG@   5: 0.401 | Hits@   5: 0.471
DCG@  10: 0.426 | Hits@  10: 0.547
DCG@ 100: 0.476 | Hits@ 100: 0.796
DCG@ 500: 0.496 | Hits@ 500: 0.947
DCG@1000: 0.502 | Hits@1000: 1.000


**3 итерация.**
- Токенизатор, основанный на WordPunctTokenizer
- Переход в нижний регистр 
- window=7
- min_count=5

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=7,
                 compute_loss=True)

embeddings_trained = model.wv

print('latest_training_loss', model.get_latest_training_loss())

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, lower=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

latest_training_loss 34326368.0


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.303 | Hits@   1: 0.303
DCG@   5: 0.381 | Hits@   5: 0.452
DCG@  10: 0.400 | Hits@  10: 0.512
DCG@ 100: 0.456 | Hits@ 100: 0.783
DCG@ 500: 0.477 | Hits@ 500: 0.945
DCG@1000: 0.482 | Hits@1000: 1.000


**4 итерация. Применение стемминга PorterStemmer**
- Токенизатор, основанный на регулярных выражениях
- Переход в нижний регистр 
- window=7
- min_count=5

In [ ]:
def question_to_vec_stem(question, embeddings, tokenizer, stem_alg, dim=200, lower=True):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    sum_question_vec = np.zeros(dim)
    cnt_token = 0
    tokens = tokenizer.get_tokens(question, lower)
    tokens = list(map(stem_alg.stem, tokens))
    
    for token in tokens:
      if embeddings.__contains__(token):
        sum_question_vec += embeddings[token]
        cnt_token += 1
    if cnt_token:
      sum_question_vec /= cnt_token
    
    return  sum_question_vec

In [ ]:
def rank_candidates_stem(question, candidates, embeddings, tokenizer, stem_alg, dim=200, lower=True):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    question_copy = deepcopy(question)
    main_vec = question_to_vec_stem(question_copy, embeddings, tokenizer, stem_alg, dim=200, lower=lower)
    lst_candidates = []
    for ind, val in enumerate(candidates):
      candidate_copy = deepcopy(val)
      candidate_vec = question_to_vec_stem(candidate_copy, embeddings, tokenizer, stem_alg, dim=200, lower=lower)
      metric = cosine_similarity(main_vec.reshape(1,-1), candidate_vec.reshape(1,-1))
      lst_candidates.append((ind, val, metric))

    lst_candidates.sort(key=lambda x: 1-x[2])
    result = []
    for tup in lst_candidates:
        result.append((tup[0], tup[1]))

    return result

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
# stem_words = [list(map(ps.stem, lst_words)) for lst_words in words]
stem_re_words = [list(map(ps.stem, lst_words)) for lst_words in re_words]
                   
from gensim.models import Word2Vec
model = Word2Vec(stem_re_words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=7,
                 compute_loss=True)

embeddings_trained = model.wv

print('latest_training_loss', model.get_latest_training_loss())

latest_training_loss 32257478.0


In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates_stem(q, ex, embeddings_trained, retokenizer, ps, lower=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.361 | Hits@   1: 0.361
DCG@   5: 0.433 | Hits@   5: 0.497
DCG@  10: 0.457 | Hits@  10: 0.571
DCG@ 100: 0.507 | Hits@ 100: 0.818
DCG@ 500: 0.526 | Hits@ 500: 0.958
DCG@1000: 0.530 | Hits@1000: 1.000


**5 итерация. Применение стемминга SnowballStemmer**
- Токенизатор, основанный на регулярных выражениях
- Переход в нижний регистр
- удаление стоп-слов
- window=10
- min_count=10

In [ ]:
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
ps = SnowballStemmer('english')
# stem_words = [list(map(ps.stem, lst_words)) for lst_words in words]
stem_re_words = []
for lst_words in re_words:
  lst_sent = []
  for word in lst_words:
    if word not in stopWords:
      lst_sent.append(ps.stem(word))
  stem_re_words.append(lst_sent)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
for i in range(5):
  print(stem_re_words[i])

['convert', 'string', 'list', 'convert', 'googl', 'result', 'object', 'pure', 'js', 'python', 'object']
['html', '5', 'canva', 'javascript', 'use', 'make', 'interact', 'draw', 'tool', 'event', 'handl', 'geometri', 'three', 'js']
['send', 'array', 'via', 'ajax', 'fail', 'get', 'list', 'item', 'unord', 'list', 'php']
['insert', 'cookiecollect', 'cookiecontain', 'c', 'creat', 'cooki', 'string', 'send']
['updat', 'one', 'element', 'bound', 'observ', 'collect', 'wpf', 'updat', 'chang', 'list', 'item', 'list']


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(stem_re_words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=10,             # consider words that occured at least 10 times
                 window=10,
                 compute_loss=True)

embeddings_trained = model.wv

print('latest_training_loss', model.get_latest_training_loss())

latest_training_loss 23753312.0


In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates_stem(q, ex, embeddings_trained, retokenizer, ps, lower=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.446 | Hits@   1: 0.446
DCG@   5: 0.534 | Hits@   5: 0.613
DCG@  10: 0.557 | Hits@  10: 0.686
DCG@ 100: 0.601 | Hits@ 100: 0.896
DCG@ 500: 0.612 | Hits@ 500: 0.980
DCG@1000: 0.614 | Hits@1000: 1.000


**6 итерация. Применение стемминга SnowballStemmer**
- Токенизатор, основанный на регулярных выражениях
- Переход в нижний регистр
- удаление стоп-слов
- window=5
- min_count=50
- negative=3
- sg=1 (skip-gram)

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(stem_re_words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=50,             
                 window=5,
                 negative=3,
                 sg=1,
                 compute_loss=True)

embeddings_trained = model.wv

print('latest_training_loss', model.get_latest_training_loss())

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates_stem(q, ex, embeddings_trained, retokenizer, ps, lower=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

latest_training_loss 63127428.0


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.510 | Hits@   1: 0.510
DCG@   5: 0.592 | Hits@   5: 0.660
DCG@  10: 0.610 | Hits@  10: 0.717
DCG@ 100: 0.648 | Hits@ 100: 0.896
DCG@ 500: 0.658 | Hits@ 500: 0.975
DCG@1000: 0.661 | Hits@1000: 1.000


**7 итерация. Применение стемминга SnowballStemmer**
- Токенизатор, основанный на регулярных выражениях
- Переход в нижний регистр
- удаление стоп-слов
- window=20
- min_count=50

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(stem_re_words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=50,             
                 window=10,
                 negative=5,
                 sg=1,
                 compute_loss=True)

embeddings_trained = model.wv

print('latest_training_loss', model.get_latest_training_loss())

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates_stem(q, ex, embeddings_trained, retokenizer, ps, lower=True)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

latest_training_loss 71179648.0


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.509 | Hits@   1: 0.509
DCG@   5: 0.589 | Hits@   5: 0.654
DCG@  10: 0.610 | Hits@  10: 0.720
DCG@ 100: 0.647 | Hits@ 100: 0.898
DCG@ 500: 0.657 | Hits@ 500: 0.973
DCG@1000: 0.660 | Hits@1000: 1.000


**8 итерация. Применение лематизации**
- Токенизатор spacy
- Переход в нижний регистр 
- min_count=50           
- window=10
- negative=5
- sg=1

Будем использовать spacy. Для этого нужно выполнить некоторые преобразования над датасетами, чтобы подавать в spacy не список готовых токенов, а предложения

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import string as st
st.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
words = []
for sample in train_data:
  spacy_results = nlp(sample[0].lower())
  q_tokens = [token.lemma_ for token in spacy_results if not token.is_stop]
  words.append(q_tokens)

# Посмотрим первые 5 строк списков токенов
for i in range(5):
    print(words[i])

['convert', 'string', 'list', 'convert', 'google', 'result', 'object', '(', 'pure', 'js', ')', 'python', 'object']
['html', '5', 'canvas', 'javascript', 'use', 'make', 'interactive', 'drawing', 'tool', '?', 'event', 'handle', 'geometry', 'three.js', '?']
['send', 'array', 'ajax', 'fail', 'get', 'list', 'item', 'unordered', 'list', 'php']
['insert', 'cookiecollection', 'cookiecontainer', '?', 'c', '#', 'create', 'cookie', 'string', 'send']
['update', 'element', 'bound', 'observable', 'collection', 'wpf-', 'update', 'change', 'list', 'item', 'list']


In [ ]:
# Сохраним для последующего переиспользования (лематизация train_data заняла 4 часа)
import pickle
with open('/content/drive/MyDrive/DLS_NLP/1w_simple_embeddings/train_lemma.pickle', 'wb') as handle:
    pickle.dump(words, handle)

In [ ]:
# Загрузим леммы
import pickle
with open('/content/drive/MyDrive/DLS_NLP/1w_simple_embeddings/train_lemma.pickle', 'rb') as handle:
    words = pickle.load(handle)

In [ ]:
# Уберем знаки пунктуации (кроме точек, которые могут иметь совсем другой смысл)
words_lower = []
lst_punct = [i for i in st.punctuation]

for lst in words:
  lower_lst = []
  for w in lst:
    if w not in lst_punct:
      lower_lst.append(w)
  words_lower.append(lower_lst)

# Посмотрим первые 5 строк lower words
for i in range(5):
    print(words_lower[i])

['convert', 'string', 'list', 'convert', 'google', 'result', 'object', 'pure', 'js', 'python', 'object']
['html', '5', 'canvas', 'javascript', 'use', 'make', 'interactive', 'drawing', 'tool', 'event', 'handle', 'geometry', 'three.js']
['send', 'array', 'ajax', 'fail', 'get', 'list', 'item', 'unordered', 'list', 'php']
['insert', 'cookiecollection', 'cookiecontainer', 'c', 'create', 'cookie', 'string', 'send']
['update', 'element', 'bound', 'observable', 'collection', 'wpf-', 'update', 'change', 'list', 'item', 'list']


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(words_lower, # data for model to train on
                 size=200,     # embedding vector size
                 min_count=50,             
                 window=10,
                 negative=5,
                 sg=1,
                 compute_loss=True)

embeddings_trained = model.wv

print('latest_training_loss', model.get_latest_training_loss())

latest_training_loss 70503856.0


In [ ]:
# Сохраним для последующего переиспользования (не хватает оперативной памяти)
import pickle
with open('/content/drive/MyDrive/DLS_NLP/1w_simple_embeddings/embeddings_trained.pickle', 'wb') as handle:
    pickle.dump(embeddings_trained, handle)

In [ ]:
# Загрузим эмбеддинги
import pickle
with open('/content/drive/MyDrive/DLS_NLP/1w_simple_embeddings/embeddings_trained.pickle', 'rb') as handle:
    embeddings_trained = pickle.load(handle)

In [ ]:
def question_to_vec_lemma(question, embeddings, nlp_alg, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    sum_question_vec = np.zeros(dim)
    cnt_token = 0
    spacy_results = nlp_alg(question)
    tokens = [token.lemma_.lower() for token in spacy_results]
    
    for token in tokens:
      if embeddings.__contains__(token):
        sum_question_vec += embeddings[token]
        cnt_token += 1        
        
    if cnt_token:
      sum_question_vec /= cnt_token
    
    return  sum_question_vec

In [ ]:
def rank_candidates_lemma(question, candidates, embeddings, nlp_alg, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    question_copy = deepcopy(question)
    main_vec = question_to_vec_lemma(question_copy, embeddings, nlp_alg, dim=200)
    lst_candidates = []
    for ind, val in enumerate(candidates):
      candidate_copy = deepcopy(val)
      candidate_vec = question_to_vec_lemma(candidate_copy, embeddings, nlp_alg, dim=200)
      metric = cosine_similarity(main_vec.reshape(1,-1), candidate_vec.reshape(1,-1))
      lst_candidates.append((ind, val, metric))

    lst_candidates.sort(key=lambda x: 1-x[2])
    result = []
    for tup in lst_candidates:
        result.append((tup[0], tup[1]))

    return result

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates_lemma(q, ex, embeddings_trained, nlp, dim=200)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.503 | Hits@   1: 0.503
DCG@   5: 0.583 | Hits@   5: 0.652
DCG@  10: 0.602 | Hits@  10: 0.710
DCG@ 100: 0.637 | Hits@ 100: 0.879
DCG@ 500: 0.649 | Hits@ 500: 0.966
DCG@1000: 0.652 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


**Какой принцип токенизации даёт качество лучше и почему?**

  - В данной задаче токенизатор с понижением регистра и удалением пунктуации показал лучшее качество. Мне кажется, это связано с тем, что предложения часто начинаются с большой буквы, и 2 одинаковых слова могут иметь разный эмбеддинг, но одинаковый смысл, а знаки пунктуации в данном кейсе в основном не несут смысловой нагрузки. 

**Помогает ли нормализация слов?**
  - Да, однозначно помогает. SnowballStemmer и лематизация с помощью spacy повышают метрики качества

**Какие эмбеддинги лучше справляются с задачей и почему?**
  - Если имеется ввиду, предобученные векторные представления слов на постах Stack Overflow и обученные в процессе решения задачи, то эмбеддинги, полученные с понижением регистра, удалением пунктуации и нормализацией (SnowballStemmer, и лематизация) полученные в ходе решения задачи справляются лучше. 
  - Мне кажется, тут есть несколько причин:
      - Неизвестно, насколько данные, на которых строились эмбеддинги (предобученные) коррелируют с данными из вадидационного датасета.
      -  При расчете метрик на предобученных эмбеддингах к валидационному датасету не применялся стемминг или лематизация
      - Параметры обучения эмбеддингов в задаче подбирались с учетом метрик, рассчитанных на валидационном датасете, что, строго говоря, не верно. 

- Если говорить о моих экспериментах (в ноутбуке представлена только часть), то лучший результат (если так можно выразиться) при размере эмбеддингов = 200 получается со следующими параметрами:
    - window=5
    - min_count=50
    - negative=3
    - sg=1 (skip-gram)
    - Обработка train:
      - Понижение регистра
      - Удаление пунктуации
      - Удаление стоп-слов
      - SnowballStemmer
    - Обработка validation:
      - Понижение регистра
      - SnowballStemmer

  - Почему?:
    - О понижении регистра и удалении пунктуации я писала выше
    - Стемминг обрезает окончания, и слова с одинаковым смыслом, но разным окончанием становятся одинаково написанными, что в целом повышает метрику косинусной близости между ними. Для этого датасета подобное преобразование имеет значение
    - Удаление стоп-слов в этой задаче, на мой взгляд оправдано, потому что различные артикли и предлоги в вопросах по IT-тематике вряд ли имеют значение.
    - Увеличение min_count дало прирост метрик, как мне кажется, потому что исследуемые датасеты основаны на вопросах из узкоспециализированной области. Отдельные вопросы могут содержать специальные термины, которые редко встречаются в других вопросах, но при этом находятся в том же контексте, что и более распространненые термины.
    - window=5. В другой итерации я получила сопоставимые результаты с window=10. Думаю, что в данной задаче размер окна, выбранный между 5 и 10, не влияет на результат
    -  negative=3. В другой итерации я устанавливала negative=5 (с повышением размера окна). Мы уже удалили стоп-слова, поэтому дополнительно еще указывать модели на количество > 5 шумных слов в контексте, которое должно было выброшено из контекста, на мой-взгляд избыточно. С другой стороны, стемминг мог "создать" не интерпретируемые формы слов, которые утратили свой смысл, и их лучше выбросить. 
    - sg=1. Skp-gram восстанавливает контект по центральному слову. Поскольку мы работаем с вопросами из области IT, то контект для многих совершенно разных слов может быть очень похожим, например - "Как отсортировать массив на python?" и "Как отсортировать массив на C++?" - в этих вопросах отличается только язык программирования, но эти же языки программирования могут встречаться и в других контекстах. На мой взгляд, тут корректнее учиться от центрального слова угадывать контекст, чем наоброт. 

Я умышленно не использовала другой размер эмбеддингов при обучении. Думаю, было бы некорректно сравнивать результат с предобученными эмбеддингами, у которых размер 200. Полагаю, что dim=300 лучше бы справился с задачей.

**Почему получилось плохое качество решения задачи?**
- Думаю, потому что у меня недостаточно опыта в решении подобрых задач. Судя по результатам, которые озвучивались в чате курса, можно было добиться значительно лучших результатов.
- Размер эмбеддинга имеет значение. Скорее всего, dim=200 недостаточен и лучше было использовать dim=300
- Кроме того, я обратила внимание, что значение loss при обучении не вполне коррелирует с метриками на валидации. Возможно, это связано с наличием значительной разницы в обучающих и валидационных примерах.

**Предложите свой подход к решению задачи.**
- Применить SnowballStemmer, понизить регистр, убрать пунктуацию
- Для обучения эмбеддингов выбрать алгоритм, позволяющий увеличить количество эпох обучения (версия Word2Vec в Колабе не поддерживает параметр epochs).
- Выбрать размер эмбеддингов = 300. 
